In [ ]:
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd
import json
from tqdm import tqdm

In [ ]:
def get_total_number_of_product_reviews(product_URL) : 
    dom = requests.get(product_URL)
    soup = BeautifulSoup(dom.text, "html.parser")
    result = soup.select_one("#li_review_text > a > span").text
    return int(result[1 : -1].replace(',', ''))

In [ ]:
def get_product_number(product_URL) : 
    start_num = product_URL.find("i_sProductcd=")
    product_number = product_URL[start_num + len("i_sProductcd=") : ]
    return product_number

In [ ]:
def scrape_product_reviews(product_URL) : 
    df = pd.DataFrame(columns = ["user_id", "channel", "option", "review", "membership"])

    base_URL = 'http://www.aritaum.com/rnl/us/rnl_us_prodUseReviewList_ajax.do?'
    
    product_number = get_product_number(product_URL)
    # Your header information
    headers={'X-Requested-With': 'XMLHttpRequest',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.84 Safari/537.36',
            'Cookie': 'WMONID=VvSHpknJ9-y; org.springframework.web.servlet.i18n.CookieLocaleResolver.LOCALE=ko; noMbrPcId=201808061103160396324; optimizelyEndUserId=oeu1533521275630r0.9608343355438038; RB_PCID=1533521276784304874; _ga=GA1.2.1217811574.1533521278; RB_GUID=60fb2f62-f1e6-41e1-add9-2760ad972ee0; s_cc=true; s_fid=0BCEA2E054A695AF-058F2959B015720A; scs=%7B%22t%22%3A1%7D; spUID=1531043289798945133b142.be16e545; JSESSIONID=00007Pou6Xda22XDiIZ1grtIp0p:189j2deea; _gid=GA1.2.2006749581.1533894461; s_ev8=N; s_ppn=aritaum-kr%5E%EC%87%BC%ED%95%91%5E%EC%83%81%ED%92%88%EC%83%81%EC%84%B8; _aprtetSessId=ry44l; ins-gaSSId=fb91d48f-c4e7-3659-21cc-e84b2c4aae8c_1533898063; _dc_gtm_UA-110770460-4=1; wcs_bt=s_51fdc9a13f4b:1533894577; RB_SSID=c4Ol7dW3W8; _aprtetSessPageSeq=2; insdrSV=4; s_sq=ap-w-aritaum-prd%2Cap-w-aptotal-prd%3D%2526pid%253Daritaum-kr%25255E%2525EC%252587%2525BC%2525ED%252595%252591%25255E%2525EC%252583%252581%2525ED%252592%252588%2525EC%252583%252581%2525EC%252584%2525B8%2526pidt%253D1%2526oid%253Dhttp%25253A%25252F%25252Fwww.aritaum.com%25252Fshop%25252Fpr%25252Fshop_pr_product_view.do%25253Fi_sTmType%25253DPC_HIS_BAK%252526i_sProductcd%25253DP0000000000%2526ot%253DA',
            'Referer': product_URL,
            'Accept': 'application/json, text/javascript, */*; q=0.01'
            }

    total_number_reviews = get_total_number_of_product_reviews(product_URL)
    print("total_number_reviews : ", total_number_reviews)
    for i in range(1, round(total_number_reviews/10) + 2):
        StartRownum = (i-1) * 10 + 1
        EndRownum = StartRownum + 9
        data={'i_iNowPageNo': str(i), 
        'i_iPageSize': '10', 
        'i_iEndRownum': str(EndRownum), 
        'i_iRecordCnt': total_number_reviews, 
        'i_iTotalPageCnt': round(total_number_reviews/10) + 1, 
        'i_iStartRownum': str(StartRownum), 
        'i_sProductcd': product_number, 
        'i_sProdTypecd': '0001',
        }
        # get response
        response = requests.post(url = base_URL, data=data, headers = headers)
        data = response.text
        dic = json.loads(data)

        review_list = dic['object']['useReview']['useReviewList']
        for review in review_list:
            data = {
                "user_id" : review['v_userid'], #userid
                "channel" : review['v_reg_channel'], #channel
                "option" : review['v_optionnm'], #product option
                "review" : review["v_content"], #review
                "membership" : review["v_levelnm"],
            }

            df.loc[len(df)] = data
    
    
    return df.to_excel("./" + product_number + ".xlsx", index = False)



In [ ]:
#아리따움 하트 인 밤
Heart_in_Balm = scrape_product_reviews("http://www.aritaum.com/shop/pr/shop_pr_product_view.do?i_sProductcd=P0000000000000040625")

In [ ]:
Heart_in_Balm.to_excel()

----